In [2]:
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
import numpy as np
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams, SumoCarFollowingParams
from flow.controllers import RLController, IDMController, ContinuousRouter
from gym.spaces.box import Box
from gym.spaces import Tuple
from flow.core.params import InFlows
from flow.controllers import SimLaneChangeController
from flow.networks import Network
import os
from flow.controllers.routing_controllers import ConstructionRouter
from flow.core.params import SumoLaneChangeParams

from env_constructionV4_nostate import myEnv

ADDITIONAL_ENV_PARAMS = {
    "max_accel": 1,
    "max_decel": 1,
}

# time horizon of a single rollout
HORIZON = 1500
# number of rollouts per training iteration
N_ROLLOUTS = 20
# number of parallel workers
N_CPUS = 2

# We place one autonomous vehicle and 22 human-driven vehicles in the network
vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             num_vehicles=2)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             #routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode="strategic",lcpushy=1.0),
             num_vehicles=0)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["gneE4","gneE35","gneE38"]
)

# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "gneE35": ["gneE35","gneE15","gneE13","gneE4.264","gneE4.264.110","gneE8","gneE9","gneE9.252","gneE33"],
                "gneE4": ["gneE4","gneE17","gneE11","gneE13","gneE4.264","gneE4.264.110","gneE8","gneE9","gneE9.252","gneE33"],
               "gneE8": ["gneE8","gneE9","gneE37","gneE38","gneE39","gneE4.264.110","gneE8"]
               }


inflow = InFlows()
inflow.add(veh_type="human",
           edge="gneE4",
           vehs_per_hour=1000,
            depart_lane="random",
            depart_speed="random",
            color="white")
inflow.add(veh_type="human",
           edge="gneE35",
           vehs_per_hour=5000,
            depart_lane="random",
            depart_speed="random",
            color="blue")


file_dir = "/home/llavezzo/"
net_params = NetParams(
    template="/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/constructionV4.net.xml",
    inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="construction_traffic",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.1,
        render=False,
        restart_instance=True
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=750,
        clip_actions=False,
        additional_params={
            "target_velocity": 20,
            "sort_vehicles": False,
            "max_accel": 1,
            "max_decel": 1,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [3, 3]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config




In [ ]:
alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})

2020-03-12 10:41:42,050	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-12 10:41:42,053	INFO resource_spec.py:212 -- Starting Ray with 4.54 GiB memory available for workers and up to 2.27 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-12 10:41:42,699	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_97b42628,RUNNING,


(pid=raylet) E0312 10:41:43.036575200     123 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1584024103.036538000","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=raylet) E0312 10:41:43.036950700     123 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet) E0312 10:41:43.096742600     123 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=146) E0312 10:41:44.931169400     146 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1584024104.931157900","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=14

(pid=262) E0312 10:41:57.422065300     262 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1584024117.422051600","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=262) E0312 10:41:57.422256800     262 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=262) /home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=262)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=262) /home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in

(pid=146) Warning: Vehicle 'flow_00.13' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=87.40.
(pid=262) Warning: Vehicle 'flow_00.13' performs emergency braking with decel=-19.75 wished=4.50 severity=3.39, time=82.70.
(pid=262) Warning: Vehicle 'flow_10.3' performs emergency braking with decel=-90.52 wished=4.50 severity=19.12, time=84.30.
(pid=146) Warning: Vehicle 'flow_00.14' performs emergency braking with decel=-91.16 wished=4.50 severity=19.26, time=93.90.
(pid=262) Warning: Vehicle 'flow_10.2' performs emergency braking with decel=-122.69 wished=4.50 severity=26.26, time=92.20.
(pid=146) Warning: Vehicle 'flow_10.9' performs emergency braking with decel=-82.46 wished=4.50 severity=17.32, time=100.40.
(pid=262) Warning: Vehicle 'flow_00.16' performs emergency braking with decel=-81.33 wished=4.50 severity=17.07, time=96.70.
(pid=262) Warning: Vehicle 'flow_10.24' performs emergency braking with decel=-130.18 wished=4.50 severity=27.93, time=99.60.
(pi

...

(pid=146) Warning: Vehicle 'flow_00.28' performs emergency braking with decel=-27.41 wished=4.50 severity=5.09, time=147.10.
(pid=262) Warning: Vehicle 'flow_10.10' performs emergency braking with decel=-31.15 wished=4.50 severity=5.92, time=86.50.
(pid=262) Warning: Vehicle 'rl_1' performs emergency braking with decel=-20.49 wished=4.50 severity=3.55, time=86.50.
(pid=146) Warning: Vehicle 'flow_00.18' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=148.20.
(pid=262) Warning: Vehicle 'flow_00.14' performs emergency braking with decel=-51.16 wished=4.50 severity=10.37, time=93.10.
(pid=262) Warning: Vehicle 'flow_00.15' performs emergency braking with decel=-11.90 wished=4.50 severity=1.64, time=93.60.
(pid=146) Warning: Vehicle 'flow_00.27' performs emergency braking with decel=-19.58 wished=4.50 severity=3.35, time=157.80.
(pid=262) Warning: Vehicle 'flow_10.7' performs emergency braking with decel=-14.80 wished=4.50 severity=2.29, time=104.80.
(pid=262) W

(pid=262) Warning: Vehicle 'flow_00.56' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=210.30.
(pid=146) Warning: Vehicle 'flow_10.6' performs emergency braking with decel=-13.07 wished=4.50 severity=1.90, time=93.20.
(pid=262) Warning: Vehicle 'rl_0' performs emergency braking with decel=-10.91 wished=4.50 severity=1.43, time=212.40.
(pid=146) Warning: Vehicle 'flow_10.10' performs emergency braking with decel=-98.92 wished=4.50 severity=20.98, time=98.00.
(pid=262) Warning: Vehicle 'flow_10.41' performs emergency braking with decel=-116.42 wished=4.50 severity=24.87, time=214.00.
(pid=146) Warning: Vehicle 'flow_00.18' performs emergency braking with decel=-59.97 wished=4.50 severity=12.33, time=106.10.
(pid=146) Warning: Vehicle 'flow_10.12' performs emergency braking with decel=-10.69 wished=4.50 severity=1.38, time=108.90.
(pid=262) Warning: Vehicle 'flow_10.41' performs emergency braking with decel=-18.65 wished=4.50 severity=3.15, time=219.00.
(pid=1

(pid=262) Warning: Vehicle 'flow_00.34' performs emergency braking with decel=-21.09 wished=4.50 severity=3.69, time=181.00.
(pid=262) Warning: Vehicle 'flow_10.57' performs emergency braking with decel=-130.03 wished=4.50 severity=27.90, time=194.10.
(pid=146) Warning: Vehicle 'flow_00.10' performs emergency braking with decel=-10.44 wished=4.50 severity=1.32, time=72.80.
(pid=146) Warning: Vehicle 'flow_00.9' performs emergency braking with decel=-12.05 wished=4.50 severity=1.68, time=75.20.
(pid=146) Warning: Vehicle 'flow_00.14' performs emergency braking with decel=-35.82 wished=4.50 severity=6.96, time=86.30.
(pid=146) Warning: Vehicle 'flow_00.13' performs emergency braking with decel=-15.39 wished=4.50 severity=2.42, time=88.20.
Result for PPO_myEnv-v0_97b42628:
  custom_metrics: {}
  date: 2020-03-12_10-59-36
  done: false
  episode_len_mean: 1500.0
  episode_reward_max: 14107.255359663995
  episode_reward_mean: -784890.0883690993
  episode_reward_min: -8896807.735804098
  epi

Trial name,status,loc,iter,total time (s),timesteps,reward
PPO_myEnv-v0_97b42628,RUNNING,160.39.217.237:145,1,1056.06,30000,-784890


(pid=146) Warning: Vehicle 'flow_10.5' performs emergency braking with decel=-23.62 wished=4.50 severity=4.25, time=98.00.
(pid=146) Warning: Vehicle 'flow_00.16' performs emergency braking with decel=-26.72 wished=4.50 severity=4.94, time=99.80.
(pid=146) Warning: Vehicle 'flow_10.16' performs emergency braking with decel=-133.74 wished=4.50 severity=28.72, time=101.80.
(pid=146) Warning: Vehicle 'flow_10.3' performs emergency braking with decel=-38.51 wished=4.50 severity=7.56, time=101.80.
(pid=146) Warning: Vehicle 'flow_10.2' performs emergency braking with decel=-14.63 wished=4.50 severity=2.25, time=102.70.
(pid=146) Warning: Vehicle 'flow_00.18' performs emergency braking with decel=-61.60 wished=4.50 severity=12.69, time=104.00.
(pid=146) Warning: Vehicle 'flow_10.15' performs emergency braking with decel=-35.25 wished=4.50 severity=6.83, time=106.50.
(pid=146) Warning: Vehicle 'flow_00.17' performs emergency braking with decel=-36.38 wished=4.50 severity=7.08, time=108.20.
(p